In [ ]:
# !pip3 install --upgrade --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# !gcloud auth login



You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

In [ ]:
# !pip install --upgrade vertexai

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import json
import pandas as pd
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting,GenerationConfig

In [ ]:
def generate(Prompt,system_prompt,generation_config,safety_settings ):
    vertexai.init(project="hclsw-gcp-srsch", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
        system_instruction=[system_prompt]
    )
    responses = model.generate_content(
        [Prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )
    return responses
    # for response in responses:
    #     print(response.text, end="")

In [ ]:
def prompt_template_4(objective, requirement):
    Prompt = f"""
    Please provide the Stories and their Story Points for the given description. Make sure to strictly follow the format demonstrated in the example below. Your response should be professional, clearly formatted with proper indentation, and include new lines where appropriate for readability:

    Example:
    Objective: "[Causal AI] Reports"
    Requirement: As a user, I should be able to review model results through reports generated for multiple use cases.

    Output:
    Stories for "[Causal AI] Reports"

    Story 1: Generate Reports for Multiple Use Cases

    Summary: The system should be able to generate reports for various use cases.

    Context: Users need to understand the results of the model across different scenarios. Reports provide a structured way to present this information.

    Acceptance Criteria:
    - The system should support the generation of reports for at least three distinct use cases.
    - Each report should contain relevant information specific to the use case, such as data used, model parameters, and key findings.
    - Users should be able to select the use cases for which they want to generate reports.

    Story Points: 5

    Story 2: Customize Report Content

    Summary: Users should be able to customize the content of the reports.

    Context: Different users may have specific needs and preferences for the information presented in the reports.

    Acceptance Criteria:
    - Users should be able to select which data points and visualizations are included in the report.
    - Users should be able to adjust the format and layout of the report.
    - Users should be able to export the report in different formats (e.g., PDF, CSV).

    Story Points: 8

    Story 3: Filter and Sort Report Data

    Summary: Users should be able to filter and sort the data presented in the reports.

    Context: This allows users to focus on specific aspects of the results and gain deeper insights.

    Acceptance Criteria:
    - Users should be able to filter the data based on specific criteria (e.g., date range, specific variables).
    - Users should be able to sort the data in ascending or descending order based on different columns.
    - The filtering and sorting options should be applied dynamically to the report content.

    Story Points: 3

    **Important:** When generating the stories, follow this format exactly as shown. Do not omit any section (Story, Summary, Context, Acceptance Criteria, and Story Points). Ensure the response is structured and formatted similarly for clarity and professionalism.

    Now, based on the format provided in the example above, generate the stories for the following:

    Objective: {objective}
    Requirement: {requirement}
    """

    system_prompt = """ You will receive a Requirement Description and Objective. Your task is to:
  Analyze the requirements to fully understand the scope and context.
  Break down the requirements into multiple smaller, manageable tasks, referred to as "Stories," only based on the explicitly mentioned aspects of the requirement. Do not include tasks beyond the given scope.
  Assign a Story Point to each task, reflecting the estimated effort required. Use the Fibonacci sequence values (1, 2, 3, 5, 8) to denote the complexity of each task.
  For each Story, follow these rules:
  - Provide a Summary that briefly describes the Story.
  - Include the Context to explain the purpose, background, and any necessary details.
  - Define the Acceptance Criteria to specify the conditions that must be met for the task to be considered complete.
  - Assign a Story Point to each story based on its complexity and effort.
  - Use proper indentation and new lines to ensure the response is clear, professional, and easy to read.
  Your goal is to create clear, actionable, and well-defined Stories that effectively decompose the explicitly stated requirements, without adding additional tasks."""

    return Prompt, system_prompt


In [ ]:
def inference_2(objective,requirement):

  prompt , system_prompt = prompt_template_4(objective,requirement)

  generation_config = {
    "max_output_tokens": 1024,
    "temperature": 0.5,
    "top_p": 0.95,
    "seed": 42,
  }


  safety_settings = [
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
    ]

  result = generate(prompt,system_prompt,generation_config,safety_settings)

  return result.text



In [ ]:
def inference(objective,requirement):

  prompt , system_prompt = prompt_template_4(objective,requirement)

  generation_config = {
    "max_output_tokens": 1024,
    "temperature": 0.5,
    "top_p": 0.95,
    "response_mime_type": "application/json",
    "response_schema": {
        "type": "OBJECT",
        "properties": {
            "stories": {
                "type": "ARRAY",
                "items": {
                    "type": "OBJECT",
                    "properties": {
                        "story_number": {"type": "STRING"},
                        "title": {"type": "STRING"},
                        "summary": {"type": "STRING"},
                        "context": {"type": "STRING"},
                        "acceptance_criteria": {
                            "type": "ARRAY",
                            "items": {"type": "STRING"}
                        },
                        "story_points": {"type": "STRING"}
                    }
                }
            }
        }
    },
    "seed": 42,
  }


  safety_settings = [
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
    ]

  result = generate(prompt,system_prompt,generation_config,safety_settings)

  return result.text



In [ ]:
# Process DataFrame within the given index range and save as JSON
def process_dataframe(file, start_idx, end_idx):
    df = pd.read_csv(file.name)

    # Ensure index values are within range
    start_idx = max(0, min(start_idx, len(df) - 1))
    end_idx = max(start_idx, min(end_idx, len(df)))

    # Process the specified range
    df = df.iloc[start_idx:end_idx]
    df['generated_stories'] = df.apply(lambda row: inference(row['Objective '], row['Final Requirement Description']), axis=1)

    # Convert to JSON format
    output_data = df.to_dict(orient='records')
    output_file = "processed_output.json"

    with open(output_file, "w") as f:
        json.dump(output_data, f, indent=4)

    return output_file

# Direct input processing with formatted JSON output
def process_direct_input(objective, requirement):
    stories = inference_2(objective, requirement)

    return stories




In [ ]:
# Gradio Interface
def gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Requirement Analysis And Creating Jira Stories")

        with gr.Tab("Upload File"):
            file_input = gr.File(label="Upload CSV File")
            start_idx = gr.Number(label="Start Index", value=0, interactive=True)
            end_idx = gr.Number(label="End Index", value=10, interactive=True)
            file_output = gr.File(label="Processed Output (JSON)", interactive=False)
            process_button = gr.Button("Process File")

            process_button.click(process_dataframe, inputs=[file_input, start_idx, end_idx], outputs=file_output)

        with gr.Tab("Direct Input"):
            objective_input = gr.Textbox(label="Objective", lines=2, placeholder="Enter the objective")
            requirement_input = gr.Textbox(label="Requirement", lines=5, placeholder="Enter the requirement")
            response_output = gr.Markdown(label="Generated Response")
            generate_button = gr.Button("Generate Response")

            generate_button.click(process_direct_input, inputs=[objective_input, requirement_input], outputs=response_output)
    return demo

In [ ]:
# Run the interface
demo = gradio_interface()
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c73583c9968114e3b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# experiments

In [ ]:
from IPython.display import display, JSON
answer =  inference("""Round-trip Editing with Office 365
 """ ,"""Allow editing and round-trip updates using desktop plugin and locally installed Office clients, due to MSFT limitations for browser-based editing. Out of scope: O365 Online, mobile, Mac support.
 """ )

In [ ]:
def process_dataframe(path):
    df = pd.read_csv(path)
    results = []
    for index, row in df.iterrows():
        objective = row['Objective ']
        requirement = row['Final Requirement Description']

        response = inference(objective, requirement)

        results.append({
            "Workspace Name": row["Workspace Name"],
            "Epic Name": row["Epic Name"],
            "Objective": objective,
            "Requirement": requirement,
            "Response": response
        })

    return pd.DataFrame(results)

In [ ]:
import pandas as pd

df = pd.read_csv("/content/Sampled_output.csv")

In [ ]:
df.head()

,Workspace Name,Epic Name,Objective,Final Requirement Description
0,AION,AION for Causal AI,[Causal AI] Prompt Creation \n,"As an end user, I should be able to ask my ana..."
1,AION,AION LITE,"As a System, I should be able to do the data E...",The system has the capability to perform Explo...
2,AION,AION for Causal AI,[Causal AI] Data Massaging \n,As a user I should be able to view the uploade...
3,AION,UX revamp with ReactJS,"Introduce user roles- Admin, Basic, Advanced \n",Admin- assigns user roles\n\nBasic- views feat...
4,AION,AION Autopilot Mode,"As a system, I should be able to automatically...",The system should automatically explore a rang...


In [ ]:
df_1 = df[:10]

df_1

,Workspace Name,Epic Name,Objective,Final Requirement Description
0,AION,AION for Causal AI,[Causal AI] Prompt Creation \n,"As an end user, I should be able to ask my ana..."
1,AION,AION LITE,"As a System, I should be able to do the data E...",The system has the capability to perform Explo...
2,AION,AION for Causal AI,[Causal AI] Data Massaging \n,As a user I should be able to view the uploade...
3,AION,UX revamp with ReactJS,"Introduce user roles- Admin, Basic, Advanced \n",Admin- assigns user roles\n\nBasic- views feat...
4,AION,AION Autopilot Mode,"As a system, I should be able to automatically...",The system should automatically explore a rang...
5,AION,AION Autopilot Mode,"As a system, I should be able to automatically...",The system should automatically explore a rang...
6,AION,UX revamp with ReactJS,Introduce Readability changes \n,1. Remove config download on 'trained model' s...
7,AION,AION for Causal AI,[Causal AI] Data Analysis based on user prompt...,"As a system, I should be able to perform analy..."
8,AION,UX revamp with ReactJS,Home Screen table revamp \n,1. Remove serial number\n2. Add use case ID in...
9,AION,AION for Causal AI,[Causal AI] Reports \n,"As a user, I should be able to review model re..."


In [ ]:
df_1.columns

Index(['Workspace Name', 'Epic Name', 'Objective ',
       'Final Requirement Description'],
      dtype='object')

In [ ]:
df_1['Final Requirement Description'][2]

'As a user I should be able to view the uploaded data & clean it up before proceeding to analysis.\n\nI should be able to manipulate columns, remove duplicate rows, recalculate derived columns \n'

In [ ]:
df_2 = process_dataframe(df_1)

In [ ]:
df_2['Response'][0]

1899

In [ ]:
answer

'{"stories": [{"acceptance_criteria": ["The system should allow users to edit documents using a desktop plugin installed on Windows machines.", "Changes made in the desktop plugin should be reflected in the document stored in the system.", "Users should be able to save the document after editing using the desktop plugin.", "The system should support round-trip updates, meaning changes made in the desktop plugin are reflected in the system and vice versa.", "The system should support opening documents from the system using locally installed Office clients.", "Changes made in the locally installed Office clients should be reflected in the document stored in the system.", "The system should support saving the document after editing using the locally installed Office clients.", "The system should support round-trip updates between locally installed Office clients and the system.", "The system should not support editing or round-trip updates using Office 365 Online, mobile, or Mac clients."

In [ ]:
df_2[['Requirement','Response']]


,Requirement,Response
0,"As an end user, I should be able to ask my ana...","{""stories"": [{""acceptance_criteria"": [""The sys..."
1,The system has the capability to perform Explo...,"{""stories"": [{""acceptance_criteria"": [""The sys..."
2,As a user I should be able to view the uploade...,"{""stories"": [{""acceptance_criteria"": [""Users s..."
3,Admin- assigns user roles\n\nBasic- views feat...,"{""stories"": [{""acceptance_criteria"": [""The sys..."
4,The system should automatically explore a rang...,"{""stories"": [{""acceptance_criteria"": [""The sys..."
5,The system should automatically explore a rang...,"{""stories"": [{""acceptance_criteria"": [""The sys..."
6,1. Remove config download on 'trained model' s...,"{""stories"": [{""acceptance_criteria"": [""The 'tr..."
7,"As a system, I should be able to perform analy...","{""stories"": [{""acceptance_criteria"": [""The sys..."
8,1. Remove serial number\n2. Add use case ID in...,"{""stories"": [{""acceptance_criteria"": [""The ser..."
9,"As a user, I should be able to review model re...","{""stories"": [{""acceptance_criteria"": [""The sys..."


In [ ]:
df_2['Response'][1]

'{"stories": [{"acceptance_criteria": ["The system should be able to calculate basic descriptive statistics for numerical and categorical variables (e.g., mean, median, mode, standard deviation, frequency distribution).", "The system should provide visualizations for data exploration, including histograms, scatter plots, box plots, and correlation matrices.", "The system should identify and handle missing values, outliers, and potential data inconsistencies.", "The system should provide insights and observations based on the EDA results, highlighting key trends, patterns, and relationships within the data."], "context": "Exploratory Data Analysis (EDA) is a crucial step in understanding data before building any model. It helps uncover patterns, relationships, and potential issues in the data, leading to better decision-making.", "story_number": "Story 1", "story_points": "8", "summary": "The system should perform basic statistical analysis and generate visualizations for data explorati

In [ ]:
import json
# Parse the JSON string into a Python dictionary
parsed_output = json.loads(df_2['Response'][3])

TypeError: the JSON object must be str, bytes or bytearray, not Series

In [ ]:
parsed_output

{'stories': [{'acceptance_criteria': ['The system should provide a user interface for entering analysis-related questions.',
    'The interface should be intuitive and user-friendly.',
    'The system should be able to interpret and process user queries to understand the intended analysis.',
    'The system should handle various question formats and phrasing.',
    'The system should guide the user in formulating clear and concise questions.',
    'The system should provide feedback on the clarity and completeness of the question.'],
   'context': "Users should be able to interact with the system through a dedicated interface for posing their analytical questions. This interface should facilitate the creation of effective prompts that accurately reflect the user's intent.",
   'story_number': 'Story 1',
   'story_points': '5',
   'summary': 'The system should provide a dedicated Prompt Handling Interface for users to ask analysis-related questions.',
   'title': 'Prompt Handling Interf

In [ ]:
parsed_output

{'stories': [{'summary': 'The system should be able to perform basic statistical analysis on the dataset.',
   'title': 'Story 1: Perform Basic Statistical Analysis on Dataset'},
  {'summary': 'The system should be able to create visualizations to explore the data.',
   'title': 'Story 2: Create Visualizations for Data Exploration'},
  {'summary': 'The system should be able to identify potential relationships and patterns in the data.',
   'title': 'Story 3: Identify Relationships and Patterns'}]}

In [ ]:
parsed_output['stories'][1]['acceptance_criteria']

['The system should support the use of the desktop plugin for editing documents stored in the system.',
 'The desktop plugin should be compatible with the supported versions of Microsoft Office on Windows.',
 'The system should support the use of locally installed Office clients for editing documents stored in the system.',
 'The locally installed Office clients should be compatible with the supported versions of Microsoft Office on Windows.']

In [ ]:
parsed_output['stories'][2]['acceptance_criteria']

['The system should provide clear instructions and documentation on how to use the desktop plugin.',
 'The system should provide clear instructions and documentation on how to use locally installed Office clients for editing documents.',
 'The system should offer support resources for users facing issues with the desktop plugin or locally installed Office clients.']

In [ ]:
import json
import re


# Parse the JSON string into a Python dictionary
parsed_output = json.loads(answer)

# Extract the response text
response = parsed_output["response"]

# Regular expression pattern to split the stories
pattern = r"Story (\d+): (.*?)\n+Summary: (.*?)\n+Context: (.*?)\n+Acceptance Criteria:(.*?)\n+Story Points: (\d+)"

# Find all the stories using the regular expression
stories = []
for match in re.finditer(pattern, response, re.DOTALL):
    story = {
        "story_number": match.group(1),
        "title": match.group(2),
        "summary": match.group(3),
        "context": match.group(4),
        "acceptance_criteria": [line.strip() for line in match.group(5).split("\n") if line.strip()],
        "story_points": match.group(6)
    }
    stories.append(story)

# Print the structured stories
print(json.dumps(stories, indent=2))


KeyError: 'response'

In [ ]:
def prompt_template_3(objective, requirement):
    Prompt = f"""
    Please provide the Stories and their Story Points for the given description professionally and clearly. Ensure the response has proper indentation, new lines where appropriate, and is formatted for readability. Use the provided examples as a guideline for structure and clarity:

    Example:
    Objective: "[Causal AI] Reports"
    Requirement: As a user, I should be able to review model results through reports generated for multiple use cases.

    Output:
    Stories for "[Causal AI] Reports"

    Story 1: Generate Reports for Multiple Use Cases

    Summary: The system should be able to generate reports for various use cases.

    Context: Users need to understand the results of the model across different scenarios. Reports provide a structured way to present this information.

    Acceptance Criteria:
    - The system should support the generation of reports for at least three distinct use cases.
    - Each report should contain relevant information specific to the use case, such as data used, model parameters, and key findings.
    - Users should be able to select the use cases for which they want to generate reports.

    Story Points: 5

    Story 2: Customize Report Content

    Summary: Users should be able to customize the content of the reports.

    Context: Different users may have specific needs and preferences for the information presented in the reports.

    Acceptance Criteria:
    - Users should be able to select which data points and visualizations are included in the report.
    - Users should be able to adjust the format and layout of the report.
    - Users should be able to export the report in different formats (e.g., PDF, CSV).

    Story Points: 8

    Story 3: Filter and Sort Report Data

    Summary: Users should be able to filter and sort the data presented in the reports.

    Context: This allows users to focus on specific aspects of the results and gain deeper insights.

    Acceptance Criteria:
    - Users should be able to filter the data based on specific criteria (e.g., date range, specific variables).
    - Users should be able to sort the data in ascending or descending order based on different columns.
    - The filtering and sorting options should be applied dynamically to the report content.

    Story Points: 3

    Now, based on the format provided in the example above, generate the stories for the following:

    Objective: {objective}
    Requirement: {requirement}
    """

    system_prompt = """ You will receive a Requirement Description and Objective. Your task is to:
  Analyze the requirements to fully understand the scope and context.
  Break down the requirements into multiple smaller, manageable tasks, referred to as \"Stories,\" only based on the explicitly mentioned aspects of the requirement. Do not include tasks beyond the given scope.
  Assign a Story Point to each task, reflecting the estimated effort required. Use the Fibonacci sequence values (1, 2, 3, 5, 8) to denote the complexity of each task.
  For each Story follow these rules:
  - Provide a Summary that briefly describes the Story.
  - Include the Context to explain the purpose, background, and any necessary details.
  - Define the Acceptance Criteria to specify the conditions that must be met for the task to be considered complete.
  - Assign a Story Point to each story based on its complexity and effort.
  - Use proper indentation and new lines to ensure the response is clear, professional, and easy to read.
  Your goal is to create clear, actionable, and well-defined Stories that effectively decompose the explicitly stated requirements, without adding additional tasks."""

    return Prompt, system_prompt